In [34]:
import numpy as np
import pandas as pd

In [35]:
data = pd.read_csv('D:/downloade2/dd/data/consolidated-metrics.csv')

In [36]:
# imputing the missing data with mean 
df1=data.fillna(data.mean())

In [37]:
df1.head()

,timestamp,apiserver_current_inflight_requests,apiserver_request_latencies_summary,apiserver_response_sizes_sum,apiserver_storage_transformation_latencies_microseconds_sum,aws_cpu_utilization,aws_ec2_cpuutilization_average,aws_ec2_network_out_average,aws_network_out,cgroup_manager_operation_rate,...,node_sockstat_TCP_inuse,node_sockstat_TCP_mem,node_sockstat_TCP_mem_bytes,node_sockstat_TCP_orphan,node_sockstat_TCP_tw,node_sockstat_sockets_used,node_timex_estimated_error_seconds,node_timex_frequency_adjustment_ratio,node_timex_maxerror_seconds,status
0,2020-09-18 21:45:03,0.008333,4.775001e+06,6871.300000,0.025000,13.8,0.000,0.000000,0.000000,0.145833,...,0.054167,2.233333,9147.733333,0.000000,0.983333,25.441667,7.625000e-07,0.025000,0.000291,NORMAL
1,2020-09-18 21:46:03,0.008333,3.862502e+06,7065.713485,0.025000,19.2,0.025,1752.173542,1752.173542,0.145833,...,0.050000,2.887500,11827.200000,0.000000,0.666667,25.420833,7.458333e-07,0.025000,0.000365,CPU_BURN_KANBAN_API_GATEWAY
2,2020-09-18 21:47:03,0.008333,3.862502e+06,6675.475000,0.025000,19.2,0.050,3504.347083,3504.347083,0.145833,...,0.054167,2.741667,11229.866667,0.000000,0.508333,24.612500,7.055728e-07,0.020833,0.000391,CPU_BURN_KANBAN_API_GATEWAY
3,2020-09-18 21:48:03,0.016138,2.483334e+06,6890.495833,0.000000,19.2,0.025,1752.173542,1752.173542,0.133333,...,0.004167,1.870833,7662.933333,0.000000,0.683333,19.108333,5.750000e-07,0.016667,0.000383,CPU_BURN_KANBAN_API_GATEWAY
4,2020-09-18 21:49:03,0.020305,4.500002e+06,7040.254167,0.020833,19.2,0.000,0.000000,0.000000,0.133333,...,0.004167,1.883333,7714.133333,0.004097,0.616667,19.062500,6.166667e-07,0.025000,0.000405,CPU_BURN_KANBAN_API_GATEWAY


In [38]:
data.status.value_counts()/len(data)

NORMAL                                  0.642059
NETWORK_DELAY_KANBAN_API_GATEWAY        0.150442
NETWORK_DELAY_KANBAN_COMMAND_SERVICE    0.116442
POD_KILL_KANBAN_API_GATEWAY             0.072194
CPU_BURN_KANBAN_API_GATEWAY             0.018631
Name: status, dtype: float64

In [39]:
# remove white space
df1['status'].replace('', np.nan, inplace=True)
df1.dropna(subset=['status'], inplace=True)

In [40]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4294 entries, 0 to 4293
Data columns (total 97 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   timestamp                                                    4293 non-null   object 
 1   apiserver_current_inflight_requests                          4293 non-null   float64
 2   apiserver_request_latencies_summary                          4293 non-null   float64
 3   apiserver_response_sizes_sum                                 4293 non-null   float64
 4   apiserver_storage_transformation_latencies_microseconds_sum  4293 non-null   float64
 5   aws_cpu_utilization                                          4294 non-null   float64
 6   aws_ec2_cpuutilization_average                               4292 non-null   float64
 7   aws_ec2_network_out_average                                  4292 non-null   f

In [41]:
data.shape

(4294, 97)

In [42]:
y= df1['status'].values

In [43]:
y

array(['NORMAL', 'CPU_BURN_KANBAN_API_GATEWAY',
       'CPU_BURN_KANBAN_API_GATEWAY', ..., 'NORMAL', 'NORMAL', 'NORMAL'],
      dtype=object)

In [44]:
df1.drop('status',inplace=True,axis=1)

In [45]:
df1.drop('timestamp',inplace =True,axis=1)
x=df1

In [46]:
# normalising the feature within range(0,1)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
x= scaler.fit_transform(x)
x= pd.DataFrame(x)

In [47]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

In [48]:
x_train, x_test,y_train,y_test = train_test_split(x,y,test_size=0.2, random_state=20)

In [49]:
# one hot encode outputs
#from keras.utils import np_utils
#y_train = to_categorical(y_train)
#y_test = to_categorical(y_test)
# encode class labels as interger
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
encoder = LabelEncoder()
encoder.fit(y_train)
encoded_y= encoder.transform(y_train)

dummy_y = np_utils.to_categorical(encoded_y)

In [50]:
dummy_y

array([[0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [51]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

In [52]:
model = Sequential()

model.add(Dense(500, activation = 'relu', input_dim= 95))
model.add(Dense(100,activation ='relu'))
model.add(Dense(50,activation='relu'))
model.add(Dense(5,activation='softmax'))

In [53]:
model.compile(optimizer ='adam', loss = 'categorical_crossentropy', metrics =['accuracy'])

In [54]:
model.fit(x_train,dummy_y, batch_size = 40 ,epochs=30, verbose=1)

Epoch 1/30
86/86 [==============================] - 2s 3ms/step - loss: 1.1906 - accuracy: 0.5803
Epoch 2/30
86/86 [==============================] - 0s 2ms/step - loss: 0.8251 - accuracy: 0.6797
Epoch 3/30
86/86 [==============================] - 0s 2ms/step - loss: 0.6150 - accuracy: 0.7701
Epoch 4/30
86/86 [==============================] - 0s 2ms/step - loss: 0.5118 - accuracy: 0.8205
Epoch 5/30
86/86 [==============================] - 0s 3ms/step - loss: 0.4324 - accuracy: 0.8452
Epoch 6/30
86/86 [==============================] - 0s 3ms/step - loss: 0.4316 - accuracy: 0.8442
Epoch 7/30
86/86 [==============================] - 0s 2ms/step - loss: 0.4035 - accuracy: 0.8653
Epoch 8/30
86/86 [==============================] - 0s 3ms/step - loss: 0.3674 - accuracy: 0.8707
Epoch 9/30
86/86 [==============================] - 0s 3ms/step - loss: 0.3374 - accuracy: 0.8835
Epoch 10/30
86/86 [==============================] - 0s 3ms/step - loss: 0.3181 - accuracy: 0.8848
Epoch 11/30
86/86 [

In [55]:
encoder = LabelEncoder()
encoder.fit(y_test)
encoded_y= encoder.transform(y_test)

test_dummy_y = np_utils.to_categorical(encoded_y)

In [70]:
pred_train= model.predict(x_train)
scores = model.evaluate(x_train, dummy_y, verbose=0)
print('Accuracy on training data: {}% \n Error on training data: {}'.format(scores[1], 1 - scores[1]))   
 
pred_test= model.predict(x_test)
scores2 = model.evaluate(x_test, test_dummy_y, verbose=0)
print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores2[1], 1 - scores2[1]))    

Accuracy on training data: 0.9670937657356262% 
 Error on training data: 0.03290623426437378
Accuracy on test data: 0.9185099005699158% 
 Error on test data: 0.08149009943008423


In [71]:
from sklearn.metrics import confusion_matrix

In [73]:
pred_test

array([[7.0878919e-03, 1.4060183e-02, 8.1522769e-04, 9.7799760e-01,
        3.9164974e-05],
       [2.3952832e-04, 6.8682276e-08, 1.9038791e-03, 9.9784303e-01,
        1.3390161e-05],
       [6.1607773e-15, 2.2872544e-12, 1.7488581e-24, 1.4142226e-16,
        1.0000000e+00],
       ...,
       [8.3769964e-06, 4.1738944e-04, 2.5486909e-11, 9.9957138e-01,
        2.8263403e-06],
       [1.1643664e-07, 9.9864382e-01, 1.8301274e-09, 1.3560917e-03,
        4.8119677e-09],
       [1.2542350e-03, 7.6347101e-01, 2.9253089e-05, 2.3523504e-01,
        1.0440929e-05]], dtype=float32)

In [75]:
matrix = confusion_matrix(test_dummy_y.argmax(axis=1), pred_test.argmax(axis=1))
matrix

array([[  2,   0,   0,  13,   0],
       [  0, 105,   1,  21,   0],
       [  0,   2,  75,  21,   0],
       [  0,   6,   1, 559,   0],
       [  0,   1,   0,   4,  48]], dtype=int64)

In [76]:
from sklearn.metrics import classification_report

In [77]:
print(classification_report(test_dummy_y.argmax(axis=1), pred_test.argmax(axis=1)))

              precision    recall  f1-score   support

           0       1.00      0.13      0.24        15
           1       0.92      0.83      0.87       127
           2       0.97      0.77      0.86        98
           3       0.90      0.99      0.94       566
           4       1.00      0.91      0.95        53

    accuracy                           0.92       859
   macro avg       0.96      0.72      0.77       859
weighted avg       0.92      0.92      0.91       859

